In [206]:
import csv
import numpy as np
from random import shuffle, sample, choice
from math import floor

In [258]:
fname = 'me_at_the_zoo.in'

In [259]:
ans = open(fname).read().splitlines()

## Parameters:

In [260]:
video_num, endpoint_num, request_num, cache_num, cache_size_limit = map(int, ans[0].split())

In [261]:
video_sizes = list(enumerate(map(int, ans[1].split())))

## Endpoints

In [262]:
rest = ans[2:]
endpoints = []
point = 0
for endpoint in range(endpoint_num):
    end_latent, num_connected = map(int,rest[point].split())
    caches = []
    for l in range(num_connected):
        caches.append(map(int,rest[point+l+1].split()))
    endpoints.append([end_latent, num_connected, caches])
    point += 1 + num_connected

## Requests

In [263]:
rest = rest[point:]

requests = []
for l in range(request_num):
    requests.append(map(int, rest[l].split()))

In [264]:
def fill_server(capacity, videos):
    curr = 0
    ret = set()
    indices = list(enumerate(videos))
    shuffle(indices)
    for i, size in indices:
        if curr + size > capacity:
            continue
        ret.add(i)
        curr += size
    return ret

In [265]:
def initial_guess():
    guess = []
    for c in range(cache_num):
        sh = sample(video_sizes, len(video_sizes))
        ans = []
        size = cache_size_limit
        while len(sh) > 0:
            if sh[0][1] < size:
                ans.append(sh[0][0])
                size -= sh[0][1]
            del sh[0]
        guess.append([c,ans])
    return guess

In [266]:
def mutate(instance):
    cacheToReplace = choice(instance)
    videoToReplace = choice(cacheToReplace[1])
    for l in range(mutateRetry):
        newVid = choice(video_sizes)[1]
        if newVid not in cacheToReplace and newVid != videoToReplace:
            cacheToReplace[1].remove(videoToReplace)
            cacheToReplace[1].append(newVid)
            if sum([l[1] for l in video_sizes]) <= cache_size_limit:
                return 
            cacheToReplace[1].remove(newVid)
            cacheToReplace[1].append(videoToReplace)

In [267]:
def mate(inst1, inst2):
    cut = int(min(len(inst1), len(inst2))/2.0)
    return inst1[:cut] + inst2[cut:]

In [296]:
def fit(solution):
    saved_time = 0 # in ms
    total_watches = 0
    
    for video_id, endpoint_id, times in requests:
        endpoint = endpoints[endpoint_id]
        # dc latency
        min_latency = endpoint[0]
        # iterate caches
        for cache, latency in endpoint[2]:
            if video_id in solution[cache]:
                min_latency = min(min_latency, latency)
        
        saved_latency = endpoint[0] - min_latency

#         print 'adding saved time: %i * %i' % (times, saved_latency)
        saved_time += saved_latency * times
#         print 'adding times: %i' % (times, )
        total_watches += times

    # average
    a = floor(1000.0 * saved_time / total_watches)
    print a
    return a

In [274]:
couples = 50
pop_size = couples**2
generations = 1000
mutate_prob = 0.01

In [290]:
def run_evol():
    pop = [initial_guess() for a in range(pop_size)]
    for gen in range(generations):
#         print pop
        fited = [[p,fit(p)] for i,p in enumerate(pop)]
        fited.sort(key = lambda x: x[1])
        best = fited[:20]
#         print fited[-1]
        best = [l[0] for l in best]
        new_pop = []
        for a in best:
            for b in best:
                new_pop.append(mate(a,b))
        for i,a in enumerate(new_pop):
            if np.random.rand() < mutate_prob:
                mutate(new_pop[i])
                
        pop = new_pop
        
#     print init

In [288]:
a = initial_guess()
b = initial_guess()
print a, '\n'
print b
c  

[[0, [98, 39, 35, 27]], [1, [18, 31, 42, 85, 91, 40, 16]], [2, [70, 23, 58, 95, 10, 46, 92, 5, 47, 42]], [3, [10, 38, 88, 0, 71]], [4, [89, 17, 72, 50]], [5, [1, 88, 55, 98, 22, 16]], [6, [36, 79, 61, 5]], [7, [68, 17, 40, 46, 82, 47, 42, 76, 5]], [8, [83, 57, 19, 47, 4, 16, 42]], [9, [54, 1, 17, 22, 10, 16, 40]]] 

[[0, [23, 79, 10, 82, 81, 85, 42, 40, 16]], [1, [14, 51, 88, 16]], [2, [41, 36, 47, 68, 99, 75, 42]], [3, [89, 91, 15, 5, 42, 16]], [4, [36, 39, 37, 5, 40, 42, 16]], [5, [26, 94, 57, 40, 47, 16]], [6, [84, 89, 13, 32, 47, 5]], [7, [46, 1, 9, 91, 6, 87, 5, 16, 40]], [8, [91, 85, 1, 68, 3, 27, 50, 42, 16, 40]], [9, [49, 22, 75, 27, 64, 70, 16]]]


In [295]:
run_evol()

86485.0
79113.0
97827.0
80516.0
106795.0
75173.0
93393.0
82455.0
74444.0
66943.0
98469.0
105030.0
157514.0
178684.0
73182.0
57818.0
124521.0
49705.0
65572.0
115904.0
79349.0
135404.0
140645.0
125794.0
99176.0
120673.0
89628.0
53248.0
97806.0
63377.0
79765.0
35970.0
120472.0
68527.0
66131.0
86584.0
114650.0
178769.0
126148.0
59404.0
98993.0
66437.0
44774.0
68497.0
89535.0
47713.0
42312.0
88651.0
89138.0
84362.0
73636.0
59809.0
99386.0
78196.0
54085.0
166833.0
143841.0
106696.0
90208.0
87290.0
95724.0
55737.0
98282.0
85548.0
60671.0
79061.0
119489.0
88190.0
70301.0
69190.0
139139.0
132641.0
92179.0
55937.0
149763.0
99762.0
112024.0
25440.0
68460.0
82382.0
40040.0
71479.0
43992.0
146069.0
81337.0
126908.0
70910.0
150986.0
180924.0
146256.0
89868.0
154293.0
69558.0
73387.0
92752.0
110958.0
49079.0
56807.0
123270.0
39540.0
107471.0
38443.0
44098.0
81360.0
119605.0
71362.0
98263.0
96171.0
130010.0
101844.0
114218.0
96012.0
132937.0
107599.0
57307.0
120598.0
113180.0
195763.0
152068.0
64405.0

KeyboardInterrupt: 

## Output

In [ ]:
def write_out(fout='output.in', cache_list):
    with open(fout, 'w') as f:
        f.write(len(cache_list))
